In [3]:
import kfp
from kfp import dsl
from kfp.components import create_component_from_func

In [4]:
@create_component_from_func
def generate_random_list_op() -> list:
    import random

    total = random.randint(5, 10)
    result = [i for i in range(1, total)]

    return result


@create_component_from_func
def print_op(num: int):
    print(f"{num} is Generated!")


@dsl.pipeline(
    name='Parallel pipeline',
)
def parallel_pipeline():
    _generate_random_list_op = generate_random_list_op()
    client = kfp.Client()
    client.create_run_from_pipeline_func(parallel_pipeline, arguments={})


    # random_list = generate_random_list_op().output

    # with dsl.ParallelFor(random_list) as item:
    #     print_op(item)


# if __name__ == "__main__":
    # kfp.compiler.Compiler().compile(
    #     parallel_pipeline,
    #     "./parallel_pipeline.yaml"
    # )

In [1]:
import kfp
from kfp import dsl
from kfp.aws import use_aws_secret

S3_PATH = 's3://changhyun-kubeflow/LPOINT_BIG_COMP_01_DEMO.csv'

def s3_sync():
    return kfp.dsl.ContainerOp(
        name='s3_sync',
        image='amazon/aws-cli:latest',
        command=['aws', 's3', 'sync', S3_PATH, '/tmp'],
        file_outputs={
            'data': '/tmp'
        }
    )


@dsl.pipeline(name='s3_sync_pipeline', description='s3 sync pipeline.')
def s3_sync_pipeline():
    _ = s3_sync().apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

In [ ]:
if __name__ == "__main__":
    kfp.compiler.Compiler().compile(s3_sync_pipeline, 's3_sync_pipeline.tar.gz')

In [5]:
a = s3_sync().apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))

In [6]:
a

{'ContainerOp': {'is_exit_handler': False, 'human_name': 's3_sync', 'display_name': None, 'name': 's3_sync 800007f583459da8', 'node_selector': {}, 'volumes': [], 'tolerations': [], 'affinity': {}, 'pod_annotations': {}, 'pod_labels': {}, 'num_retries': 0, 'retry_policy': None, 'backoff_factor': None, 'backoff_duration': None, 'backoff_max_duration': None, 'timeout': 0, 'init_containers': [], 'sidecars': [], 'loop_args': None, '_component_spec_inputs_with_pipeline_params': [], '_inputs': [], 'dependent_names': [], 'enable_caching': True, 'attrs_with_pipelineparams': ['node_selector', 'volumes', 'pod_annotations', 'pod_labels', 'num_retries', 'init_containers', 'sidecars', 'tolerations', '_container', 'artifact_arguments', '_parameter_arguments'], '_is_v2': False, '_container': {'args': None,
 'command': ['aws',
             's3',
             'sync',
             's3://changhyun-kubeflow/LPOINT_BIG_COMP_01_DEMO.csv',
             '/tmp'],
 'env': [{'name': 'AWS_ACCESS_KEY_ID',
         